In [1]:
import pandas as pd
import numpy as np
import xarray as xr

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
ds = xr.open_dataset('/Users/victoriaford/Desktop/Train_Files/CAHYDRO_BROC1.nc')

df = ds.to_dataframe()
df

tas  pr tas_qc  elevation      lat  \
station       time                                                          
CAHYDRO_BROC1 2010-05-30 15:30:00  300.940 NaN    nan   549.8592  33.2314   
              2010-05-31 15:20:00  301.490 NaN    nan   549.8592  33.2314   
              2010-06-05 15:20:00  303.160 NaN    nan   549.8592  33.2314   
              2010-06-06 15:25:00  304.830 NaN    nan   549.8592  33.2314   
              2010-06-12 15:25:00  294.830 NaN    nan   549.8592  33.2314   
...                                    ...  ..    ...        ...      ...   
              2022-07-31 23:50:00  311.494 NaN    nan   549.8592  33.2314   
              2022-08-06 23:20:00  315.383 NaN    nan   549.8592  33.2314   
              2022-08-08 00:40:00  312.050 NaN    nan   549.8592  33.2314   
              2022-08-21 00:20:00  310.939 NaN    nan   549.8592  33.2314   
              2022-08-22 00:30:00  312.606 NaN    nan   549.8592  33.2314   

                                        lon  
station       time                           
CAHYDRO_BROC1 2010-05-30 15:30:00 -116.4144  
              2010-05-31 15:20:00 -116.4144  
              2010-06-05 15:20:00 -116.4144  
              2010-06-06 15:25:00 -116.4144  
              2010-06-12 15:25:00 -116.4144  
...                                     ...  
              2022-07-31 23:50:00 -116.4144  
              2022-08-06 23:20:00 -116.4144  
              2022-08-08 00:40:00 -116.4144  
              2022-08-21 00:20:00 -116.4144  
              2022-08-22 00:30:00 -116.4144  

[2973 rows x 6 columns]

Motivation: individual gross outliers from general station distribution are a common error in obs data by random recording, reporting, formatting, or instrumentation errors

Process:
1. uses individual observation deviations derived from monthly mean climatology calculated for each hour of the day
2. climatologies calculated using winsorised data to remove initial effect of outliers
    - Winsorising: all values beyond threhsold value from mean are set to that threshold value
    - 5 and 95% for hadisd
    - number of data values in population remains the same, not trimmed
    - where data further from mean are removed from population
3. raw unwinsorised observations are anomalised using these climatologies
4. standardized by IQR for that month and hour
    - IQR cannot be less than 1.5degC
5. values are low-pass filtered to remove any climate change signal causing overzealous removal at ends of time series
6. gaussian is fitted to the histogram of anomalies for each month
7. threshold value, rounded outwards where crosses y=0.1 line
8. distribution beyond threhsold value is scanned for gap, equal to bin width or more
9. all values beyond gap are flagged
10. obs that fall between critical threshold value and gap or critical threshold and end of distribution are tentatively flagged
    - these may be later reinstated on comparison with good data from neighboring stations

Notes:
- when applied to SLP, frequently flags storm signals, which may be of high interest, so this test is not applied to pressure data
- hadisd only applies to temp and dewpoint temp